# Multithreading in Python

---

## Table of Contents
1. Introduction to Multithreading
2. The Global Interpreter Lock (GIL)
3. Creating Threads
4. Thread Synchronization
5. Locks, RLocks, and Semaphores
6. Conditions and Events
7. Thread Pools with concurrent.futures
8. Thread-Local Data
9. Common Patterns and Best Practices
10. Key Points
11. Practice Exercises

---

## 1. Introduction to Multithreading

**Thread**: A lightweight unit of execution within a process.

**Concurrency vs Parallelism**:
- **Concurrency**: Multiple tasks making progress (not necessarily simultaneously)
- **Parallelism**: Multiple tasks executing simultaneously (multiple CPUs)

**When to use threads**:
- I/O-bound tasks (network, file operations)
- Maintaining responsiveness in applications
- When tasks can run independently

In [1]:
import threading
import time

# Basic thread example
def print_numbers():
    for i in range(5):
        time.sleep(0.1)
        print(f"Number: {i}")

def print_letters():
    for letter in 'ABCDE':
        time.sleep(0.1)
        print(f"Letter: {letter}")

# Sequential execution
print("=== Sequential ===")
start = time.time()
print_numbers()
print_letters()
print(f"Time: {time.time() - start:.2f}s")

=== Sequential ===
Number: 0
Number: 1
Number: 2
Number: 3
Number: 4
Letter: A
Letter: B
Letter: C
Letter: D
Letter: E
Time: 1.00s


In [2]:
# Concurrent execution with threads
print("=== Concurrent ===")
start = time.time()

t1 = threading.Thread(target=print_numbers)
t2 = threading.Thread(target=print_letters)

t1.start()  # Start thread 1
t2.start()  # Start thread 2

t1.join()   # Wait for thread 1 to complete
t2.join()   # Wait for thread 2 to complete

print(f"Time: {time.time() - start:.2f}s")

=== Concurrent ===
Number: 0
Letter: A
Number: 1
Letter: B
Number: 2
Letter: C
Letter: D
Number: 3
Number: 4
Letter: E
Time: 0.51s


---

## 2. The Global Interpreter Lock (GIL)

The GIL is a mutex that protects access to Python objects, preventing multiple threads from executing Python bytecode simultaneously.

**Implications**:
- Only one thread executes Python code at a time
- CPU-bound tasks don't benefit from threading
- I/O-bound tasks DO benefit (GIL is released during I/O)

In [3]:
# CPU-bound task - threading doesn't help
def cpu_bound(n):
    """CPU-intensive calculation."""
    count = 0
    for i in range(n):
        count += i * i
    return count

# Sequential
start = time.time()
cpu_bound(5000000)
cpu_bound(5000000)
print(f"Sequential CPU-bound: {time.time() - start:.2f}s")

# Threaded (not faster due to GIL)
start = time.time()
t1 = threading.Thread(target=cpu_bound, args=(5000000,))
t2 = threading.Thread(target=cpu_bound, args=(5000000,))
t1.start(); t2.start()
t1.join(); t2.join()
print(f"Threaded CPU-bound: {time.time() - start:.2f}s")

Sequential CPU-bound: 2.09s
Threaded CPU-bound: 2.11s


In [4]:
# I/O-bound task - threading helps!
def io_bound(seconds):
    """I/O-bound operation (simulated with sleep)."""
    time.sleep(seconds)
    return f"Slept for {seconds}s"

# Sequential
start = time.time()
io_bound(0.5)
io_bound(0.5)
print(f"Sequential I/O-bound: {time.time() - start:.2f}s")

# Threaded (much faster!)
start = time.time()
t1 = threading.Thread(target=io_bound, args=(0.5,))
t2 = threading.Thread(target=io_bound, args=(0.5,))
t1.start(); t2.start()
t1.join(); t2.join()
print(f"Threaded I/O-bound: {time.time() - start:.2f}s")

Sequential I/O-bound: 1.00s
Threaded I/O-bound: 0.50s


---

## 3. Creating Threads

In [5]:
# Method 1: Using target function
def worker(name, count):
    for i in range(count):
        print(f"{name}: {i}")
        time.sleep(0.1)

# Create thread with arguments
thread = threading.Thread(
    target=worker,
    args=("Thread-A", 3),
    name="MyWorker"  # Optional thread name
)

print(f"Thread name: {thread.name}")
print(f"Is alive: {thread.is_alive()}")

thread.start()
print(f"Is alive: {thread.is_alive()}")

thread.join()
print(f"Is alive: {thread.is_alive()}")

Thread name: MyWorker
Is alive: False
Thread-A: 0
Is alive: True
Thread-A: 1
Thread-A: 2
Is alive: False


In [6]:
# Method 2: Subclassing Thread
class WorkerThread(threading.Thread):
    def __init__(self, name, count):
        super().__init__()
        self.name = name
        self.count = count
        self.result = None

    def run(self):
        """Override run() method."""
        total = 0
        for i in range(self.count):
            total += i
            time.sleep(0.05)
        self.result = total

worker = WorkerThread("Calculator", 10)
worker.start()
worker.join()
print(f"Result: {worker.result}")

Result: 45


In [7]:
# Thread attributes and methods
def demo_thread():
    current = threading.current_thread()
    print(f"Current thread: {current.name}")
    print(f"Thread ID: {current.ident}")
    print(f"Is daemon: {current.daemon}")
    time.sleep(0.1)

# Main thread info
main_thread = threading.main_thread()
print(f"Main thread: {main_thread.name}")
print(f"Active threads: {threading.active_count()}")

t = threading.Thread(target=demo_thread, name="DemoThread")
t.start()
t.join()

print(f"All threads: {threading.enumerate()}")

Main thread: MainThread
Active threads: 5
Current thread: DemoThread
Thread ID: 136892988261952
Is daemon: False
All threads: [<_MainThread(MainThread, started 136893572689920)>, <Thread(IOPub, started daemon 136893404530240)>, <Heartbeat(Heartbeat, started daemon 136893396137536)>, <ControlThread(Control, started daemon 136893370959424)>, <ParentPollerUnix(Thread-2, started daemon 136893005047360)>]


In [8]:
# Daemon threads
# Daemon threads are terminated when main program exits

def daemon_worker():
    while True:
        print("Daemon working...")
        time.sleep(0.5)

def normal_worker():
    for i in range(3):
        print(f"Normal: {i}")
        time.sleep(0.3)

daemon = threading.Thread(target=daemon_worker, daemon=True)
normal = threading.Thread(target=normal_worker)

daemon.start()
normal.start()

normal.join()  # Wait for normal thread
print("Normal thread done - daemon will be killed")
# Daemon continues until notebook cell completes

Daemon working...
Normal: 0
Normal: 1
Daemon working...
Normal: 2
Normal thread done - daemon will be killed


---

## 4. Thread Synchronization

Race conditions occur when threads access shared data concurrently.

In [9]:
# Race condition example
class Counter:
    def __init__(self):
        self.value = 0

    def increment(self):
        # This is NOT atomic!
        # read -> modify -> write can be interleaved
        current = self.value
        time.sleep(0.0001)  # Simulate some processing
        self.value = current + 1

def worker_unsafe(counter, times):
    for _ in range(times):
        counter.increment()

# Without synchronization
counter = Counter()
threads = [threading.Thread(target=worker_unsafe, args=(counter, 100)) for _ in range(5)]

for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Expected: 500, Actual: {counter.value}")

Expected: 500, Actual: 105


In [10]:
# Fixed with Lock
class SafeCounter:
    def __init__(self):
        self.value = 0
        self.lock = threading.Lock()

    def increment(self):
        with self.lock:  # Only one thread at a time
            current = self.value
            time.sleep(0.0001)
            self.value = current + 1

def worker_safe(counter, times):
    for _ in range(times):
        counter.increment()

counter = SafeCounter()
threads = [threading.Thread(target=worker_safe, args=(counter, 100)) for _ in range(5)]

for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Expected: 500, Actual: {counter.value}")

Daemon working...
Expected: 500, Actual: 500


---

## 5. Locks, RLocks, and Semaphores

In [11]:
# Lock - basic mutual exclusion
lock = threading.Lock()

# Method 1: acquire/release
lock.acquire()
try:
    # Critical section
    print("Lock held")
finally:
    lock.release()

# Method 2: context manager (preferred)
with lock:
    print("Lock held with context manager")

# Non-blocking acquire
if lock.acquire(blocking=False):
    try:
        print("Got the lock!")
    finally:
        lock.release()
else:
    print("Lock was busy")

# Acquire with timeout
if lock.acquire(timeout=1.0):
    try:
        print("Got lock within timeout")
    finally:
        lock.release()

Lock held
Lock held with context manager
Got the lock!
Got lock within timeout


In [12]:
# RLock - reentrant lock (can be acquired multiple times by same thread)
rlock = threading.RLock()

def recursive_function(n):
    with rlock:
        if n > 0:
            print(f"Level {n}")
            recursive_function(n - 1)

# This would deadlock with a regular Lock!
recursive_function(3)
print("Completed without deadlock")

Level 3
Level 2
Level 1
Completed without deadlock


In [13]:
# Semaphore - allows N threads to access resource simultaneously
semaphore = threading.Semaphore(3)  # Max 3 concurrent

def limited_resource(name):
    print(f"{name} waiting for semaphore...")
    with semaphore:
        print(f"{name} acquired semaphore")
        time.sleep(0.5)  # Simulate work
        print(f"{name} releasing semaphore")

threads = [threading.Thread(target=limited_resource, args=(f"Thread-{i}",))
           for i in range(6)]

for t in threads:
    t.start()
for t in threads:
    t.join()

print("All threads completed")

Thread-0 waiting for semaphore...Thread-1 waiting for semaphore...
Thread-1 acquired semaphore

Thread-0 acquired semaphore
Thread-2 waiting for semaphore...
Thread-2 acquired semaphore
Thread-3 waiting for semaphore...
Thread-4 waiting for semaphore...
Thread-5 waiting for semaphore...
Daemon working...
Thread-1 releasing semaphore
Thread-0 releasing semaphore
Thread-2 releasing semaphore
Thread-4 acquired semaphore
Thread-3 acquired semaphore
Thread-5 acquired semaphore
Daemon working...
Thread-4 releasing semaphore
Thread-3 releasing semaphore
Thread-5 releasing semaphore
All threads completed


In [14]:
# BoundedSemaphore - raises error if released more than acquired
bounded_sem = threading.BoundedSemaphore(2)

bounded_sem.acquire()
bounded_sem.release()

try:
    bounded_sem.release()  # Error! Released more than acquired
except ValueError as e:
    print(f"Error: {e}")

Error: Semaphore released too many times


---

## 6. Conditions and Events

In [15]:
# Condition - wait for a condition to be true
condition = threading.Condition()
data_ready = False
shared_data = None

def producer():
    global data_ready, shared_data
    time.sleep(0.5)  # Simulate preparation

    with condition:
        shared_data = "Important Data"
        data_ready = True
        print("Producer: Data is ready")
        condition.notify_all()  # Wake up all waiting threads

def consumer(name):
    global data_ready, shared_data

    with condition:
        while not data_ready:
            print(f"{name}: Waiting for data...")
            condition.wait()  # Release lock and wait
        print(f"{name}: Received '{shared_data}'")

# Reset
data_ready = False
shared_data = None

threads = [
    threading.Thread(target=consumer, args=("Consumer-1",)),
    threading.Thread(target=consumer, args=("Consumer-2",)),
    threading.Thread(target=producer),
]

for t in threads:
    t.start()
for t in threads:
    t.join()

Consumer-1: Waiting for data...
Consumer-2: Waiting for data...
Daemon working...
Producer: Data is ready
Consumer-1: Received 'Important Data'
Consumer-2: Received 'Important Data'


In [16]:
# Event - simple flag for signaling between threads
event = threading.Event()

def waiter(name):
    print(f"{name}: Waiting for event...")
    event.wait()  # Block until event is set
    print(f"{name}: Event received!")

def setter():
    time.sleep(0.5)
    print("Setter: Setting event")
    event.set()  # Wake up all waiters

event.clear()  # Reset event

threads = [
    threading.Thread(target=waiter, args=("Waiter-1",)),
    threading.Thread(target=waiter, args=("Waiter-2",)),
    threading.Thread(target=setter),
]

for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Event is set: {event.is_set()}")

Waiter-1: Waiting for event...
Waiter-2: Waiting for event...
Daemon working...
Setter: Setting event
Waiter-1: Event received!
Waiter-2: Event received!
Event is set: True


In [17]:
# Event with timeout
timeout_event = threading.Event()

def wait_with_timeout():
    result = timeout_event.wait(timeout=0.3)
    if result:
        print("Event was set!")
    else:
        print("Timeout - event was not set")

thread = threading.Thread(target=wait_with_timeout)
thread.start()
thread.join()

Timeout - event was not set


In [18]:
# Barrier - synchronization point for multiple threads
barrier = threading.Barrier(3)  # Wait for 3 threads

def worker(name):
    print(f"{name}: Working...")
    time.sleep(0.1 * hash(name) % 5)  # Variable work time
    print(f"{name}: Waiting at barrier")
    barrier.wait()  # Block until all 3 threads arrive
    print(f"{name}: Passed barrier!")

threads = [threading.Thread(target=worker, args=(f"Thread-{i}",)) for i in range(3)]

for t in threads:
    t.start()
for t in threads:
    t.join()

Thread-0: Working...
Thread-1: Working...
Thread-2: Working...
Daemon working...
Daemon working...
Daemon working...
Daemon working...
Daemon working...
Daemon working...
Thread-0: Waiting at barrier
Daemon working...
Daemon working...
Thread-1: Waiting at barrier
Thread-2: Waiting at barrier
Thread-2: Passed barrier!
Thread-0: Passed barrier!
Thread-1: Passed barrier!


---

## 7. Thread Pools with concurrent.futures

In [19]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def download_page(url):
    """Simulate downloading a page."""
    time.sleep(0.2)  # Simulate network delay
    return f"Content from {url}"

urls = [f"http://example.com/page{i}" for i in range(5)]

# Using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=3) as executor:
    # Submit all tasks
    futures = [executor.submit(download_page, url) for url in urls]

    # Get results as they complete
    for future in as_completed(futures):
        result = future.result()
        print(result)

Daemon working...
Content from http://example.com/page0
Content from http://example.com/page1
Content from http://example.com/page2
Content from http://example.com/page3
Content from http://example.com/page4


In [20]:
# Using map() for simpler cases
def process_item(item):
    time.sleep(0.1)
    return item ** 2

items = list(range(10))

with ThreadPoolExecutor(max_workers=4) as executor:
    # map() preserves order
    results = list(executor.map(process_item, items))

print(f"Results: {results}")

Daemon working...
Results: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [21]:
# Handling exceptions
def risky_operation(n):
    if n == 3:
        raise ValueError(f"Error with {n}")
    time.sleep(0.1)
    return n * 2

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = {executor.submit(risky_operation, i): i for i in range(5)}

    for future in as_completed(futures):
        n = futures[future]
        try:
            result = future.result()
            print(f"Item {n}: {result}")
        except Exception as e:
            print(f"Item {n} failed: {e}")

Item 0: 0
Item 3 failed: Error with 3
Item 1: 2
Item 2: 4
Item 4: 8


In [22]:
# Future methods and callbacks
def slow_computation(x):
    time.sleep(0.2)
    return x * x

def callback(future):
    print(f"Callback: Result is {future.result()}")

with ThreadPoolExecutor(max_workers=2) as executor:
    future = executor.submit(slow_computation, 5)

    # Add callback
    future.add_done_callback(callback)

    # Check status
    print(f"Running: {future.running()}")
    print(f"Done: {future.done()}")

    # Wait for result
    result = future.result(timeout=5)
    print(f"Final result: {result}")

Running: True
Done: False
Daemon working...
Callback: Result is 25Final result: 25



---

## 8. Thread-Local Data

In [23]:
# Thread-local storage - each thread has its own copy
thread_local = threading.local()

def worker(name):
    # Each thread has its own 'value'
    thread_local.value = name
    time.sleep(0.1)
    # No interference from other threads
    print(f"Thread {name}: value = {thread_local.value}")

threads = [threading.Thread(target=worker, args=(f"T{i}",)) for i in range(3)]

for t in threads:
    t.start()
for t in threads:
    t.join()

Thread T0: value = T0
Thread T1: value = T1
Thread T2: value = T2


In [24]:
# Practical example: database connection per thread
class DatabaseConnection:
    def __init__(self, thread_name):
        self.thread_name = thread_name

    def query(self, sql):
        return f"{self.thread_name} executed: {sql}"

connection_holder = threading.local()

def get_connection():
    if not hasattr(connection_holder, 'conn'):
        thread_name = threading.current_thread().name
        connection_holder.conn = DatabaseConnection(thread_name)
    return connection_holder.conn

def database_worker():
    conn = get_connection()  # Gets or creates thread-local connection
    result = conn.query("SELECT * FROM users")
    print(result)

threads = [
    threading.Thread(target=database_worker, name=f"Worker-{i}")
    for i in range(3)
]

for t in threads:
    t.start()
for t in threads:
    t.join()

Worker-0 executed: SELECT * FROM users
Worker-1 executed: SELECT * FROM users
Worker-2 executed: SELECT * FROM users


---

## 9. Common Patterns and Best Practices

In [25]:
# Producer-Consumer pattern with Queue
from queue import Queue

def producer(queue, items):
    for item in items:
        time.sleep(0.1)  # Simulate work
        queue.put(item)
        print(f"Produced: {item}")
    queue.put(None)  # Sentinel to signal end

def consumer(queue):
    while True:
        item = queue.get()
        if item is None:
            break
        time.sleep(0.15)  # Simulate processing
        print(f"Consumed: {item}")
        queue.task_done()

q = Queue()
items = ['A', 'B', 'C', 'D', 'E']

prod = threading.Thread(target=producer, args=(q, items))
cons = threading.Thread(target=consumer, args=(q,))

prod.start()
cons.start()

prod.join()
cons.join()

Produced: A
Produced: B
Consumed: A
Daemon working...
Produced: C
Consumed: B
Produced: D
Produced: E
Consumed: C
Consumed: D
Daemon working...
Consumed: E


In [26]:
# Multiple producers and consumers
from queue import Queue
import random

def multi_producer(queue, name, count):
    for i in range(count):
        time.sleep(random.uniform(0.05, 0.15))
        item = f"{name}-{i}"
        queue.put(item)
        print(f"Produced: {item}")

def multi_consumer(queue, name, stop_event):
    while not stop_event.is_set():
        try:
            item = queue.get(timeout=0.1)
            time.sleep(random.uniform(0.05, 0.1))
            print(f"{name} consumed: {item}")
            queue.task_done()
        except:
            pass  # Timeout, check stop_event

q = Queue()
stop = threading.Event()

producers = [
    threading.Thread(target=multi_producer, args=(q, f"P{i}", 3))
    for i in range(2)
]
consumers = [
    threading.Thread(target=multi_consumer, args=(q, f"C{i}", stop))
    for i in range(3)
]

for c in consumers:
    c.start()
for p in producers:
    p.start()

for p in producers:
    p.join()

q.join()  # Wait until all items processed
stop.set()  # Signal consumers to stop

for c in consumers:
    c.join()

print("All done!")

Produced: P1-0
Produced: P0-0
C0 consumed: P1-0
Produced: P1-1
C1 consumed: P0-0
Produced: P0-1
C2 consumed: P1-1
Produced: P1-2
C0 consumed: P0-1
Produced: P0-2
C1 consumed: P1-2
Daemon working...
C0 consumed: P0-2
All done!


In [27]:
# Reader-Writer Lock pattern
class ReadWriteLock:
    """Multiple readers OR single writer."""

    def __init__(self):
        self._read_ready = threading.Condition(threading.Lock())
        self._readers = 0

    def acquire_read(self):
        with self._read_ready:
            self._readers += 1

    def release_read(self):
        with self._read_ready:
            self._readers -= 1
            if self._readers == 0:
                self._read_ready.notify_all()

    def acquire_write(self):
        self._read_ready.acquire()
        while self._readers > 0:
            self._read_ready.wait()

    def release_write(self):
        self._read_ready.release()

rw_lock = ReadWriteLock()
shared_data = {"value": 0}

def reader(name):
    rw_lock.acquire_read()
    try:
        print(f"{name} reading: {shared_data['value']}")
        time.sleep(0.1)
    finally:
        rw_lock.release_read()

def writer(name, value):
    rw_lock.acquire_write()
    try:
        print(f"{name} writing: {value}")
        shared_data['value'] = value
        time.sleep(0.1)
    finally:
        rw_lock.release_write()

threads = [
    threading.Thread(target=reader, args=("R1",)),
    threading.Thread(target=reader, args=("R2",)),
    threading.Thread(target=writer, args=("W1", 42)),
    threading.Thread(target=reader, args=("R3",)),
]

for t in threads:
    t.start()
    time.sleep(0.05)
for t in threads:
    t.join()

R1 reading: 0
R2 reading: 0
W1 writing: 42
R3 reading: 42


In [28]:
# Best practices summary
"""
1. Always use 'with' for locks to ensure release
2. Minimize the critical section (time holding lock)
3. Use thread-safe data structures (Queue, etc.)
4. Prefer ThreadPoolExecutor over manual thread management
5. Be careful with daemon threads (may not cleanup properly)
6. Use events/conditions instead of polling
7. Consider using multiprocessing for CPU-bound tasks
8. Always handle exceptions in threads
9. Use thread-local storage for per-thread state
10. Avoid shared mutable state when possible
"""

# Example of clean thread management
class ManagedThread:
    def __init__(self, name):
        self.name = name
        self._stop_event = threading.Event()
        self._thread = None

    def start(self):
        self._stop_event.clear()
        self._thread = threading.Thread(target=self._run, name=self.name)
        self._thread.start()

    def stop(self):
        self._stop_event.set()
        if self._thread:
            self._thread.join()

    def _run(self):
        while not self._stop_event.is_set():
            print(f"{self.name}: Working...")
            self._stop_event.wait(0.2)  # Sleep with ability to wake up
        print(f"{self.name}: Stopped cleanly")

worker = ManagedThread("Worker")
worker.start()
time.sleep(0.5)
worker.stop()

Worker: Working...
Daemon working...
Worker: Working...
Worker: Working...
Worker: Stopped cleanly


---

## 10. Key Points

1. **GIL**: Limits CPU-bound parallelism, I/O-bound benefits from threads
2. **Thread Creation**: Use `Thread(target=func)` or subclass `Thread`
3. **Lock**: Basic mutual exclusion, use `with lock:` syntax
4. **RLock**: Reentrant lock for recursive functions
5. **Semaphore**: Limit concurrent access to N threads
6. **Event**: Simple signaling between threads
7. **Condition**: Wait for complex conditions
8. **Barrier**: Synchronize multiple threads at a point
9. **ThreadPoolExecutor**: High-level thread management
10. **Queue**: Thread-safe producer-consumer communication

---

## 11. Practice Exercises

In [29]:
# Exercise 1: Create a thread-safe bounded buffer
# - Fixed size buffer
# - put() blocks if full
# - get() blocks if empty
# - Use Condition for synchronization

class BoundedBuffer:
    pass

# Test:
# buffer = BoundedBuffer(3)
# buffer.put(1)
# buffer.put(2)
# print(buffer.get())  # 1
# print(buffer.get())  # 2

In [30]:
# Exercise 2: Implement a thread-safe singleton
# - Only one instance should ever exist
# - Must be thread-safe

class ThreadSafeSingleton:
    pass

# Test with multiple threads trying to create instances

Daemon working...


In [31]:
# Exercise 3: Create a periodic task executor
# - Runs a function at fixed intervals
# - Can be started and stopped
# - Non-blocking start/stop

class PeriodicTask:
    pass

# Test:
# task = PeriodicTask(lambda: print("Tick"), interval=0.5)
# task.start()
# time.sleep(2)
# task.stop()

In [32]:
# Exercise 4: Implement a thread pool from scratch
# - Fixed number of worker threads
# - submit(func, *args) method
# - shutdown() method

class SimpleThreadPool:
    pass

# Test:
# pool = SimpleThreadPool(3)
# pool.submit(print, "Hello")
# pool.submit(print, "World")
# pool.shutdown()

In [33]:
# Exercise 5: Create a parallel map function
# - parallel_map(func, items, num_threads=4)
# - Returns results in order
# - Handles exceptions

def parallel_map(func, items, num_threads=4):
    pass

# Test:
# results = parallel_map(lambda x: x**2, range(10), num_threads=3)
# print(results)  # [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

---

## Solutions

In [34]:
# Solution 1:
class BoundedBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.condition = threading.Condition()

    def put(self, item):
        with self.condition:
            while len(self.buffer) >= self.capacity:
                self.condition.wait()
            self.buffer.append(item)
            self.condition.notify()

    def get(self):
        with self.condition:
            while len(self.buffer) == 0:
                self.condition.wait()
            item = self.buffer.pop(0)
            self.condition.notify()
            return item

buffer = BoundedBuffer(3)
buffer.put(1)
buffer.put(2)
buffer.put(3)
print(f"Get: {buffer.get()}")
print(f"Get: {buffer.get()}")

Get: 1
Get: 2


In [35]:
# Solution 2:
class ThreadSafeSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls):
        if cls._instance is None:
            with cls._lock:
                # Double-check locking
                if cls._instance is None:
                    cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self):
        self.value = "Singleton"

# Test
instances = []

def create_singleton():
    instances.append(ThreadSafeSingleton())

threads = [threading.Thread(target=create_singleton) for _ in range(10)]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"All same instance: {all(i is instances[0] for i in instances)}")

All same instance: True


In [36]:
# Solution 3:
class PeriodicTask:
    def __init__(self, func, interval):
        self.func = func
        self.interval = interval
        self._stop_event = threading.Event()
        self._thread = None

    def _run(self):
        while not self._stop_event.is_set():
            self.func()
            self._stop_event.wait(self.interval)

    def start(self):
        if self._thread is None or not self._thread.is_alive():
            self._stop_event.clear()
            self._thread = threading.Thread(target=self._run)
            self._thread.start()

    def stop(self):
        self._stop_event.set()
        if self._thread:
            self._thread.join()

task = PeriodicTask(lambda: print("Tick"), interval=0.3)
task.start()
time.sleep(1)
task.stop()
print("Task stopped")

Tick
Tick
Daemon working...
Tick
Tick
Daemon working...
Task stopped


In [37]:
# Solution 4:
from queue import Queue

class SimpleThreadPool:
    def __init__(self, num_workers):
        self.tasks = Queue()
        self.workers = []
        self._shutdown = False

        for _ in range(num_workers):
            worker = threading.Thread(target=self._worker)
            worker.daemon = True
            worker.start()
            self.workers.append(worker)

    def _worker(self):
        while True:
            task = self.tasks.get()
            if task is None:
                break
            func, args, kwargs = task
            try:
                func(*args, **kwargs)
            except Exception as e:
                print(f"Task error: {e}")
            self.tasks.task_done()

    def submit(self, func, *args, **kwargs):
        self.tasks.put((func, args, kwargs))

    def shutdown(self, wait=True):
        if wait:
            self.tasks.join()
        for _ in self.workers:
            self.tasks.put(None)
        for worker in self.workers:
            worker.join()

pool = SimpleThreadPool(3)
for i in range(5):
    pool.submit(print, f"Task {i}")
pool.shutdown()
print("Pool shutdown")

Task 0Task 1
Task 2
Task 3
Task 4

Pool shutdown


In [38]:
# Solution 5:
from concurrent.futures import ThreadPoolExecutor

def parallel_map(func, items, num_threads=4):
    results = [None] * len(items)
    errors = {}

    def worker(index, item):
        try:
            results[index] = func(item)
        except Exception as e:
            errors[index] = e

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [
            executor.submit(worker, i, item)
            for i, item in enumerate(items)
        ]
        # Wait for all to complete
        for future in futures:
            future.result()

    if errors:
        raise Exception(f"Errors in items: {errors}")

    return results

results = parallel_map(lambda x: x**2, range(10), num_threads=3)
print(f"Results: {results}")

Results: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
